# Heston model for the volatility surface
#### Roland Grinis - MIPT & GrinisRIT

This is a stochastic volatility model for the forward dynamics in the risk neutral measure, and CIR dynamics for the variance:
$$
\begin{split}
\text{d}f_t & =  \sqrt{\alpha_t} f_t \text{d}W_t^1  \\
\text{d}\alpha_t & =  \kappa (\theta - \alpha_t)\text{d}t + \xi \sqrt{\alpha_t} \left( \rho \text{d}W_t^1 + \sqrt{1-\rho^2} \text{d}W_t^2 \right)  
\end{split}
$$

The vol correlation obeys $-1 < \rho < 1$  as previously, and we impose the Feller condition:
$$
2\kappa\theta > \xi ^2
$$
to ensure that the volatility $\sigma_t$ stays strictly positive.

The forward price of a call option is given by:
$$
FV_C(t_0, T, K) = f_{t_0} H(\sqrt{-1}) - K \cdot H(0) 
$$
where:
$$
H(z) = \frac{1}{2} + \frac{1}{\pi} \int^{\infty}_0 \mathbf{Re} \left[ \frac{e^{u\sqrt{-1} \log \frac{f_{t_0}}{K}}}{u\sqrt{-1}} \phi(u-z,T) \right]\text{d}u
$$
with 
$$
\phi(u,T) = \exp \left( - \frac{\kappa\bar{\alpha}\rho T u\sqrt{-1} }{\nu} - A + \frac{2 \kappa \bar{\alpha} }{\nu^2}D\right)
$$
where $A = A_1 / A_2$:
$$
A_1 = (u^2 + u\sqrt{-1}) \sinh \frac{\lambda T}{2}, \quad \lambda = \left( \xi^2 + \nu^2 (u^2 + u\sqrt{-1})\right)^{\frac{1}{2}}, \quad \xi = \kappa - \nu\rho u\sqrt{-1} 
$$
and 
$$
\log A_2 = \frac{\lambda T}{2} + \log \left( \frac{\lambda +\xi}{2 \alpha_{t_0}} + \frac{\lambda  - \xi}{2\alpha_{t_0}}e^{-\lambda T}\right)
$$
with
$$
D = \log \frac{\lambda}{\alpha_{t_0}} + \frac{\kappa T}{2} - \log A_2
$$
The integral above can be truncated to $(0,200)$ and evaluated via Gaussian quadrature at 64 nodes for $10^{-10}$ accuracy.

# Heston Simulation

$$
\begin{aligned}
d \ln X(t) & =-\frac{1}{2} V(t) d t+\sqrt{V(t)} d W_X(t) \\
d V(t) & =\kappa(\theta-V(t)) d t+\varepsilon \sqrt{V(t)} d W_V(t)
d W_X(t) d W_X(t) = 0
\end{aligned}
$$

### Euler Discretization Scheme

$$
\begin{aligned}
\ln \hat{X}(t+\Delta) & =\ln \hat{X}(t)-\frac{1}{2} \hat{V}(t) \Delta+\sqrt{\hat{V}(t)} Z_X \sqrt{\Delta}, \\
\hat{V}(t+\Delta) & =V(t)+\kappa(\theta-\hat{V}(t)) \Delta+\varepsilon \sqrt{\hat{V}(t)} Z_V \sqrt{\Delta},
\end{aligned}
$$

where $Z_X$ and $Z_V$ are standardized Gaussian variables with correlation $\rho$. So they can be computed as follows:

$$
\begin{aligned}
& Z_V=\Phi^{-1}\left(U_1\right), \\
& Z_X=\rho Z_V+\sqrt{1-\rho^2} \Phi^{-1}\left(U_2\right).
\end{aligned}
$$

($U_1$ and $U_2$ are independent uniform samples, and $Φ^{−1}$ is the inverse cumulative
Gaussian distribution function).

However the discrete process for $V$ can become negative with non-zero probability, which in turn would make computation of $\sqrt{\hat{V}}$ impossible and cause the time-stepping scheme to fail. The scheme that appears to produce the smallest discretization bias can be written on the form:

$$
\begin{aligned}
\ln \hat{X}(t+\Delta) & =\ln \hat{X}(t)-\frac{1}{2} \hat{V}(t)^{+} \Delta+\sqrt{\hat{V}(t)^{+}} Z_X \sqrt{\Delta} \\
\hat{V}(t+\Delta) & =\hat{V}(t)+\kappa\left(\theta-\hat{V}(t)^{+}\right) \Delta+\varepsilon \sqrt{\hat{V}(t)^{+}} Z_V \sqrt{\Delta},
\end{aligned}
$$
where we use the notation $x^{+}=\max (x, 0)$

### Bias free Scheme

To obtain a bias-free scheme for sampling the asset price process, first integrate the SDE for $V(t)$, to yield
$$
V(t+\Delta)=V(t)+\int_t^{t+\Delta} \kappa(\theta-V(u)) d u+\varepsilon \int_t^{t+\Delta} \sqrt{V(u)} d W_V(u)
$$
or, equivalently,
$$
\int_t^{t+\Delta} \sqrt{V(u)} d W_V(u)=\varepsilon^{-1}\left(V(t+\Delta)-V(t)-\kappa \theta \Delta+\kappa \int_t^{t+\Delta} V(u) d u\right)
$$
A Cholesky decomposition shows that
$$
d \ln X(t)=-\frac{1}{2} V(t) d t+\rho \sqrt{V(u)} d W_V(u)+\sqrt{1-\rho^2} \sqrt{V(u)} d W(u)
$$
where $W$ is a Brownian motion independent of $W_V$. In integral form,
$$
\begin{aligned}
\ln X(t+\Delta)=\ln X(t)+\frac{\rho}{\varepsilon} & (V(t+\Delta)-V(t)-\kappa \theta \Delta) \\
& +\left(\frac{\kappa \rho}{\varepsilon}-\frac{1}{2}\right) \int_t^{t+\Delta} V(u) d u+\sqrt{1-\rho^2} \int_t^{t+\Delta} \sqrt{V(u)} d W(u)
\end{aligned}
$$

As $W$ is independent of $V$, conditional on $V(t)$ and $\int_t^{t+\Delta} V(u) d u$, the Ito integral $\int_t^{t+\Delta} \sqrt{V(u)} d W(u)$ is Gaussian with mean zero and variance $\int_t^{t+\Delta} V(u) d u$.

### Process for $V$ (CIR)

Non-central chi-squared distribution:
$$
\mathbb{P}\left(V_{t \cdot \Delta}<V \mid V_{t}\right) \quad = \quad F_{\chi^{12}}\left(\frac{4 \kappa V}{\varepsilon^{2}\left(1-e^{-\kappa \Delta}\right)} ; \frac{4 \kappa \theta}{\varepsilon^{2}}, \frac{4 V_{t} \kappa e^{-\kappa \Delta}}{\varepsilon^{2}\left(1-e^{-\kappa \Delta}\right)}\right) \quad
$$


$$
\begin{aligned}
m = \mathbb{E}\left( V_{t+\Delta} \mid V_t \right) \quad & = \quad \theta+(\hat{V}(t)-\theta) e^{-\kappa \Delta} \\
s^2 = Var\left( V_{t+\Delta} \mid V_t \right) \quad & = \quad \frac{\hat{V}(t) \varepsilon^2 e^{-\kappa \Delta}}{\kappa}\left(1-e^{-\kappa \Delta}\right)+\frac{\theta \varepsilon^2}{2 \kappa}\left(1-e^{-\kappa \Delta}\right)^2
\end{aligned}
$$


### Quadratic exponential (QE) approximation

Switching rule:
- $ V_t \gg 1: \quad V_{t+\Delta} \approx a \left( b + z^v \right)^2 \text{( where } z^v \text{ from } N(0,1)) $
- $V_t \ll 1: \quad f_{V_{t+\Delta}}(v) \approx p \delta(v) + \beta(1-\rho) e^{-\beta v} $

PDF with fast sampling (for $f_{V_{t+\Delta}}(v)$):
$$
\begin{aligned}
F_{V_{t+\Delta}}^{-1}(u) & \approx\left\{\begin{array}{l}
0\ ,\ 0 \leq u \leq p \\
\beta^{-1} \ln \left(\frac{1-p}{1-u}\right)\ ,\ p \leq u \leq 1
\end{array}\right. \\
u & \sim u[0,1]-\text { uniform }
\end{aligned}
$$

### Generating random numbers

Pseudo-random integers:
$$
0 \leq N_k \leq N \quad, \quad N_{k+1} = f(N_k) \equiv N
$$

$$
u \sim u[0,1] \quad , \quad u_k = \frac{N_k}{N}
$$

Box-Muller method:
$$ u_1 u_2 \sim u[0,1] \quad \theta = 2\pi u_2 \quad p = \sqrt{-2 \ln{u_1}} $$

$$ Z_1 = p\cos{\theta} $$
$$ Z_2 = p\sin{\theta} $$
Then $Z_1$ and $Z_2$ are independent random variables with a standard normal distribution $N(0,1).$


### Computing parameters $(a,\ b,\ p,\ \beta)$

$\psi \equiv \frac{S^{2}}{m^{2}} \quad \text { if } \psi \leqslant 2$:
$$
\begin{array}{l}
b^{2}=2 \psi^{-1}-1+\sqrt{2 \psi^{-1}} \sqrt{2 \psi^{-1}-1} \geq 0 \\
a=\frac{m}{1+b^{2}} \rightarrow \\
\rightarrow \text { 2nd moment matching}
\end{array}
$$

if $\psi \geqslant 1$:
$$
\begin{array}{l}
p=\frac{\psi-1}{\psi+2} \in[0,1) \\
\beta=\frac{1-p}{m}=\frac{2}{m(1+\psi)} \rightarrow \\
\rightarrow \text { moment matching at order 2}
\end{array}
$$

Switching rule: $\psi_{c} \in[1,2].$

$$
\begin{array}{l}
Y_{t_{j+1}}^{k}=\hat{V}_{t_{j+1}}^{k}-\hat{V}_{t_{j}}^{k}, \quad \Delta_{1}=t_{j+1}-t_{1}, \quad \hat{V}_{t_{N}}^{k}=\sum_{j=1}^{N} \hat{Y}_{t_{j}}^{k} \\
\Delta=t_{N}-t_{0} ; \quad k=1, \ldots, M \\
\hat{m}_{T}=\frac{1}{M} \sum_{k} \hat{V}_{t_{N}}^{k} \\
\hat{S}_{T}^{2}=\frac{1}{M-1} \sum_{k}\left(\hat{V}_{t_{N}}^{k}-\hat{m}\right)^{2}
\end{array}
$$

### Moment matching technique

$$
\begin{array}{l}
\hat{m}_{T} \quad \quad m_T = \mathbb{E}[V_T \mid V_{t_0}] \\
\hat{S}^{2}_T = S^{2}_T \quad \quad S^{2}_T = Var[V_T \mid V_{t_0}] \rightarrow \\
\rightarrow V^{k}_T = m_T + \frac{S_T}{\hat{S}_T} (\hat{V}{k}_T - \hat{m}_T) = \sum_{j=1}^{N} \left(\frac{m_T}{N} + \frac{S_T}{\hat{S}_T} \left(\hat{Y}^{k}_{t_j} - \frac{m_T}{N}\right)\right)
\end{array}
$$


### Simulating $\ln{X_{t\Delta}}$

Euler Scheme for $\ln{X_{t\Delta}}$ with QE sampling for $V_{t+\Delta}$ leads to CORRELATION LEAKS

Exact simulation is too heavy:
$$
\begin{array}{l}
 \int_{t}^{t+\Delta} V_{u} d u \approx \gamma_{1} V_{t}+\gamma_{2} V_{t+\Delta} \quad \begin{array}{l}
\gamma_{1}+\gamma_{2}=1 \\
\gamma_{1}, \gamma_{2} \geqslant 0
\end{array} \\
\end{array}
$$

Regularity condition:
$$
\mathbb{E}\left(x_{t \cdot \Delta} \mid x_{t}\right)<\infty \Longrightarrow \frac{\rho}{\varepsilon}\left(1+\kappa \gamma_{2} \Delta\right)-\frac{1}{2} \gamma_{2} \Delta \rho^{2}<\frac{2 k}{\varepsilon^{2} (1-e^{-\kappa \Delta})}
$$

Natural discretization scheme:
$$
\begin{aligned}
\ln \hat{X}(t+\Delta)= & \ln \hat{X}(t)+\frac{\rho}{\varepsilon}(\hat{V}(t+\Delta)-\hat{V}(t)-\kappa \theta \Delta)+\Delta\left(\frac{\kappa \rho}{\varepsilon}-\frac{1}{2}\right)\left(\gamma_1 \hat{V}(t)+\gamma_2 \hat{V}(t+\Delta)\right) \\
& +\sqrt{\Delta} \sqrt{1-\rho^2} \sqrt{\gamma_1 \hat{V}(t)+\gamma_2 \hat{V}(t+\Delta)} \cdot Z \\
= & \ln \hat{X}(t)+K_0+K_1 \hat{V}(t)+K_2 \hat{V}(t+\Delta)+\sqrt{K_3 \hat{V}(t)+K_4 \hat{V}(t+\Delta)} \cdot Z
\end{aligned}
$$
where $Z$ is a standard Gaussian random variable, independent of $\hat{V}$, and $K_0, \ldots, K_4$ are given by
$$
\begin{aligned}
& K_0=-\frac{\rho \kappa \theta}{\varepsilon} \Delta, \quad K_1=\gamma_1 \Delta\left(\frac{\kappa \rho}{\varepsilon}-\frac{1}{2}\right)-\frac{\rho}{\varepsilon} \\
& K_2=\gamma_2 \Delta\left(\frac{\kappa \rho}{\varepsilon}-\frac{1}{2}\right)+\frac{\rho}{\varepsilon}, \quad K_3=\gamma_1 \Delta\left(1-\rho^2\right), \quad K_4=\gamma_2 \Delta\left(1-\rho^2\right) .
\end{aligned}
$$



### Enforcing Martingale property $\mathbb{E}\left(X_{t+\Delta} \mid X_{t}\right)=X_{t} $:

$$
\begin{array}{l}
K_{0}^{*}=\left\{\begin{array}{l}
-\frac{A B^{2} a}{1-2 A a}+\frac{1}{2} \ln (1-2 A a)-\left(K_{1}+\frac{1}{2} \Delta \gamma^{\prime}\right), \quad \psi \leq \psi_{c} \\
-\ln \left(\frac{\beta(1-\rho)}{\beta-A}\right)-\left(K_{1}+\frac{1}{2} \Delta \gamma_{1}\right), \quad \psi_{>} \psi_{c} \\
\end{array}\right.
\end{array}
$$
$$
A=\frac{\rho}{\varepsilon^{2}}\left(1+K \gamma_{2} \Delta\right)-\frac{1}{2} \gamma_{2} \Delta\rho^{2}
$$

### Variance reduction: antithetic variates

$$
\begin{array}{l}
u \sim u(0,1) \longrightarrow 1-u \sim u(0,1) \longrightarrow \hat{V}=\mathbb{E}\left[P\left(V_{T}\right)\right] \\
Z \sim N(0,1) \longrightarrow-Z \sim N(0,1) \longrightarrow V^{-} \\
V_{A V}=\frac{1}{2}\left(\hat{V}+V^{-}\right) \\
\operatorname{Var}\left[V_{A V}\right]=\frac{1}{4} \operatorname{Var}\left[\hat{V}+V^{-}\right]=\frac{1}{4} \operatorname{Var} \hat{V}+\frac{1}{4} \operatorname{Var} V^{-}+\frac{1}{2} \operatorname{Cov}\left(\hat{V}, V^{-}\right) \\
\left|\operatorname{cov}\left(\hat{V}, V^{-}\right)\right| \leqslant \frac{1}{2} \operatorname{Var} \hat{V}+\frac{1}{2} \operatorname{Var} V^{-} \Rightarrow \\
 \Rightarrow \operatorname{Var V}_{A V} \leqslant \operatorname{Var} \hat{V} \text{ - negative correlation effect for antithetics}
\end{array}
$$

### Variance reduction: control variates
$$
\begin{array}{l}
\hat{V} = \mathbb{E}[P(V_T)] \\
\text{C – analytic formula (like a vanilla call)} \\
\hat{C} = \mathbb{E}[P_C(V_T)] \\
V = \hat{V} + \alpha (C - \hat{C}) \\
\end{array}
$$


# Longstaff-Schwarz Method (LSM algorythm)

#### Dynamic programming:
$$
\begin{array}{l}
V_{T_M} = \psi \quad - \quad \text{Payoff} \ \\
C_{T_n} = \mathbb{E}[V_{T_{n+1}}] \quad - \quad \text{(Analytic/numerical integration, trees, PDEs, Monte-Carlo...)}\\
V_{T_n} = max[C_{T_n}, \psi] \\
\end{array}
$$

#### Monte-Carlo + Regression (LSM)

- Simulate N paths $f_{n,k} = f_k(T_n)$, $f_{n,k} \leq K$ – keep contract if ITM;
- Make $g_k = \psi(f_{m,k})$ – cash-flow vector;
- Recursively fit $C_{T_n} = \sum_{l} b_{n,l} \phi_l$ on exercised paths, if $\psi(f_{n,k}) > C_{T_n}(f_{n,k})$ then update $g_k = \psi(f_{n,k})$;
(${\phi_l}$ is basis for $C$, e.g. monomials);
- Calculate $V_0 = \frac{1}{N} \sum_{k} g_k$.

### References

* Cui, Y. et al. *Full and fast calibration of the Heston stochastic volatility model.* European Journal of Operational Research 263.2 (2017)
* Grzelak, L.A. et al. *Mathematical Modeling and Computation in Finance: With Exercises and Python and MATLAB Computer Codes*, World Scientific Pub. (2019)